In [10]:
dir_files <- "/Users/imrankabir/Desktop/research/vqa_accessibility/result_generator/data/result_april_2/output/csv/"
out_dir <- "/Users/imrankabir/Desktop/research/vqa_accessibility/result_generator/results/r_tables/"

for (file in list.files(dir_files)){
    data <- read.csv(paste(dir_files, file, sep=""))

    gt <- data$gt.cosine.similarity
    y_vqa <- data$VQA.cosine.similarity
    y_resnet152 <- data$resnet.152.feature.similarity

    pairs <- list(
        list('gpv vs gt', gt, y_vqa),
        list('resnet152 vs gt', gt, y_resnet152)
    )

    row <- c('')
    titles <- c("Title")
    correlations <- c("Correlation")
    f_statistics <- c("F Statistics")
    multiple_r_squareds <- c("Multiple R-squareds")
    p_values <- c("P Values")
    coeff_interps <- c("Coefficent (intercept Pr(>|t|))")
    coeff_row_2s <- c("Coefficent (Row 2 Pr(>|t|))")

    for (i in 1:length(pairs)){
        pair = pairs[[i]]
        title <- pair[[1]]
        x <- pair[[2]]
        y <- pair[[3]]

        model <- lm(y~x)

        f_stat_val <- unname(summary(model)$fstatistic[1])
        numdf <- unname(summary(model)$fstatistic[2])
        dendf <- unname(summary(model)$fstatistic[3])

        corr <- cor(y, x)
        f_stat <- sprintf("%s on %s DoF", f_stat_val, dendf)
        r_squared <- summary(model)$r.squared
        p_val <- anova(model)$'Pr(>F)'[1]
        coeff_interp <- summary(model)$coefficients[1,"Pr(>|t|)"]
        coeff_row_2 <- summary(model)$coefficients[2,"Pr(>|t|)"]

        row <- c(row, i)
        titles <- c(titles, title)
        correlations <- c(correlations, corr)
        f_statistics <- c(f_statistics, f_stat)
        multiple_r_squareds <- c(multiple_r_squareds, r_squared)
        p_values <- c(p_values, p_val)
        coeff_interps <- c(coeff_interps, coeff_interp)
        coeff_row_2s <- c(coeff_row_2s, coeff_row_2)
    }

    df <- data.frame (
        row,
        titles,
        correlations,
        f_statistics,
        multiple_r_squareds,
        p_values,
        coeff_interps,
        coeff_row_2s
    )
    write.table(
        df,
        paste(out_dir, file, sep=""),
        sep=",", row.names=FALSE, col.names=FALSE
    )
}
